In [28]:
import numpy as np
import pandas as pd
import math

In [29]:
data=pd.read_excel('oil3.xlsx')
kij=pd.read_excel('kpdf.xlsx')

In [30]:
data

,z,Tc,Pc,w,P,T,A1,0.533272,Unnamed: 8,−1247.4,...,Unnamed: 12,5.71419,Unnamed: 14,3.83354,Unnamed: 16,Unnamed: 17,Unnamed: 18,θ,Unnamed: 20,Unnamed: 21
0,0.0016,32.980,12.93,-0.217000,100.0,377.55,A2,0.000191017,NaN,0.792,...,NaN,2.715790,NaN,1.19629,NaN,NaN,NaN,0.771819,15.687500,NaN
1,0.0091,304.200,72.80,0.225000,NaN,NaN,A3,7.79681e-08,NaN,1971,...,NaN,0.286590,NaN,34.8888,NaN,NaN,NaN,14.700000,13.472100,NaN
2,0.3647,190.560,45.99,0.011000,NaN,NaN,A4,-2.8437e-11,NaN,27000,...,NaN,39.854400,NaN,36.1952,NaN,NaN,NaN,5.927140,0.435770,NaN
3,0.0967,282.340,50.41,0.087000,NaN,NaN,A5,9594679999999999883717115904,NaN,707.4,...,NaN,0.122488,NaN,104.193,NaN,NaN,NaN,6.096480,-1.062843,NaN
4,0.0695,364.900,46.00,0.142000,NaN,NaN,A6,NaN,NaN,NaN,...,NaN,24.752200,NaN,α,NaN,NaN,NaN,1.288620,-1.492099,NaN
5,0.0144,407.850,36.40,0.186000,NaN,NaN,Tbc,992.353,NaN,NaN,...,NaN,35.315500,NaN,0.228181,NaN,NaN,NaN,0.169347,0.712314,ω
6,0.0393,425.120,37.96,0.200000,NaN,NaN,TC,1285.73,to,714.296,...,NaN,218.000000,NaN,211.207,NaN,NaN,NaN,15.251800,NaN,NaN
7,0.0144,460.390,33.81,0.229000,NaN,NaN,NaN,Rankine,NaN,Kelvin,...,NaN,5.384495,NaN,psia,to,14.56,bar,NaN,NaN,NaN
8,0.0141,469.700,33.70,0.252000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,992.352895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.3762,714.296,14.56,0.712314,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
kij

,H2,CO2,C1,C2,C3,IC4,C4,IC5,C5,C6+,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,0,0,0.000000,0.000000,0.000000,0,0,0,0,0.000,H2,NaN,NaN,NaN,0.134090,NaN,NaN
1,0,0,0.000000,0.000000,0.000000,0,0,0,0,0.000,CO2,NaN,NaN,NaN,2.285430,NaN,c1c6
2,0,0,0.000000,-0.121232,-0.105856,0,0,0,0,0.135,C1,NaN,NaN,NaN,7.614550,NaN,0.00776585
3,0,0,-0.121232,0.000000,0.000000,0,0,0,0,0.000,C2,NaN,NaN,NaN,10.465650,NaN,-0.121232
4,0,0,-0.105856,0.000000,0.000000,0,0,0,0,0.000,C3,NaN,NaN,NaN,5.235100,NaN,-0.105856
5,0,0,0.000000,0.000000,0.000000,0,0,0,0,0.000,IC4,NaN,NaN,w,0.712314,NaN,NaN
6,0,0,0.000000,0.000000,0.000000,0,0,0,0,0.000,C4,NaN,NaN,NaN,0.087000,NaN,NaN
7,0,0,0.000000,0.000000,0.000000,0,0,0,0,0.000,IC5,NaN,NaN,NaN,0.142000,NaN,NaN
8,0,0,0.000000,0.000000,0.000000,0,0,0,0,0.000,C5,NaN,NaN,NaN,NaN,NaN,NaN
9,0,0,0.135000,0.000000,0.000000,0,0,0,0,0.000,C6+,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
#Import shift volume parameters
s=[0,0,-0.154,-0.1002,-0.08501,-0.07935,-0.06413,-0.0435,-0.04183,0.153896837]

In [33]:
T=data.iat[0,5]
P=data.iat[0,4] #bar#
R=8.314472*10**(-5) #m3*bar/mol*K
Tc=np.array(data['Tc'])
Pc=np.array(data['Pc'])
w=np.array(data['w'])
z=np.array(data['z'])
Pb=80
aL=pd.DataFrame(np.zeros((len(Tc),len(Tc))))
aV=pd.DataFrame(np.zeros((len(Tc),len(Tc))))

In [34]:
K=[]
y=[]
for i in range (0,len(Tc)):
    K=np.append(K,(Pc[i]/Pb)*math.exp(5.37*(1+w[i])*(1-Tc[i]/T))) #estimating K values from Wilson's Correlation
    y=np.append(y,K[i]*z[i])

In [35]:
#Bubble point calculation
SSF=12
n=0
while  SSF>10**(-18):
    if n>=1:
        K=[]
        y=[]
        for i in range (0,len(Tc)):
            K=np.append(K,φLi[i]/φVi[i])
            y=np.append(y,K[i]*z[i])
    α=[]
    bLi=[]
    bL=[]
    bVi=[]
    bV=[]
    cor1=[]
    for i in range (0,len(Tc)):  
        bLi=np.append(bLi,(0.0778*R*Tc[i])/Pc[i])
        bVi=np.append(bVi,(0.0778*R*Tc[i])/Pc[i])
        bL=np.append(bL,z[i]*bLi[i])
        bV=np.append(bV,y[i]*bVi[i])
        cor1=np.append(cor1,s[i]*bL[i])
        if w[i]<=0.491:
            m=0.37464+1.54226*w[i]-0.26992*w[i]*w[i]
            α=np.append(α,(1/Pc[i])*0.45724*(R*R)*(Tc[i]*Tc[i])*(1+m*(1-math.sqrt(T/Tc[i])))**2)
        else:
            m=0.379642+1.48503*w[i]-0.164423*w[i]*w[i]+0.016666*w[i]*w[i]*w[i]
            α=np.append(α,(1/Pc[i])*0.45724*(R*R)*(Tc[i]*Tc[i])*(1+m*(1-math.sqrt(T/Tc[i])))**2)
    for i in range (0,len(Tc)): 
        for j in range(0,len(Tc)):
            aL.iat[i,j]=z[i]*z[j]*(1-kij.iat[i,j])*math.sqrt(α[i]*α[j])
            aV.iat[i,j]=y[i]*y[j]*(1-kij.iat[i,j])*math.sqrt(α[i]*α[j])
    amL=aL.sum().sum()
    amV=aV.sum().sum()
    bmL=np.sum(bL)
    bmV=np.sum(bV)
    AL =(amL*Pb)/(R*T)**2
    AV=(amV*Pb)/(R*T)**2
    BL=(bmL*Pb)/(R*T)
    BV=(bmV*Pb)/(R*T)
    ZroL=np.array([1,BL-1,AL-2*BL-3*(BL*BL),(BL**3)+(BL**2)-AL*BL])
    ZroL=np.roots(ZroL)
    RealL=[]
    for i in range(len(ZroL)):
            if abs(ZroL[i].imag)<10**(-5):
                RealL=np.append(RealL, np.real(ZroL[i]))
    ZL=min(RealL[RealL>0])          
    ZroV=np.array([1,BV-1,AV-2*BV-3*(BV*BV),(BV**3)+(BV**2)-AV*BV])
    ZroV=np.roots(ZroV)
    RealV=[]
    for i in range(len(ZroV)):
            if abs(ZroV[i].imag)<10**(-5):
                RealV=np.append(RealV, np.real(ZroV[i]))
    ZV=max(RealV[RealV>0])
    lnφLi=[]
    φLi=[]
    fLi=[]
    lnφVi=[]
    φVi=[]
    fVi=[]
    SF=[]
    g1V=[]
    g1L=[]
    for i in range (0,len(Tc)):
        gL=[]
        gV=[]
        for j in range (0,len(Tc)):
            gL=np.append(gL,z[j]*math.sqrt(α[j])*(1-kij.iat[i,j]))
            gV=np.append(gV,y[j]*math.sqrt(α[j])*(1-kij.iat[i,j]))
        g1L=np.append(g1L,sum(gL))
        g1V=np.append(g1V,sum(gV))
        lnφL=-math.log(ZL-BL)+(ZL-1)*(bLi[i]/bmL)-(AL/(BL*(2**1.5)))*((2*math.sqrt(α[i])*g1L[i]/amL)-(bLi[i]/bmL))*math.log((ZL+BL*(2**0.5+1))/(ZL-BL*(2**0.5-1)))
        lnφV=-math.log(ZV-BV)+(ZV-1)*(bVi[i]/bmV)-(AV/(BV*(2**1.5)))*((2*math.sqrt(α[i])*g1V[i]/amV)-(bVi[i]/bmV))*math.log((ZV+BV*(2**0.5+1))/(ZV-BV*(2**0.5-1)))
        lnφLi=np.append(lnφLi,lnφL)
        lnφVi=np.append(lnφVi,lnφV)
        φLi=np.append(φLi,math.exp(lnφLi[i]))
        φVi=np.append(φVi,math.exp(lnφVi[i]))
        fLi=np.append(fLi,φLi[i]*Pb*z[i])   
        fVi=np.append(fVi,φVi[i]*Pb*y[i])  
        SF=np.append(SF,(1-(fLi[i]/fVi[i]))**2)
    SSF=sum(SF)
    Pb=Pb*sum(K*z)
    n=n+1

In [36]:
print ('Bubble point pressure is up to',Pb,'bar')

Bubble point pressure is up to 195.48644047605288 bar


In [37]:
component=np.array(kij['Unnamed: 10'])
x=pd.DataFrame(y,columns=['vapor mole fraction'])
x['component']=component
x

,vapor mole fraction,component
0,0.004672,H2
1,0.009653,CO2
2,0.787454,C1
3,0.113338,C2
4,0.050326,C3
5,0.006922,IC4
6,0.016631,C4
7,0.004572,IC5
8,0.004102,C5
9,0.002331,C6+


In [41]:
#volume of the system at saturation pressure using PR Equation of State
VsatPR=ZL*R*T/Pb
print (Vsat, 'cubic meters/mole')

0.00017746572865900506 cubic meters/mole


In [43]:
#saturation volume prediction by using volume translation as suggested by Jhaveri and Youngren
VsatJPR=Vsat-sum(cor1)
print (VsatJPR, 'cubic meters/mole')

0.00016162606845688388 cubic meters/mole
